In [1]:
import pandas as pd
from utils import _convert_to_ffm

In [2]:
data = {
    'Click' : [0, 1, 0],
    'Advertiser': ['Nike', 'ESPN', 'Nike'],
    'Publisher': ['CNN', 'BBC', 'BBC'],
    'Numeric': [10, 20, 80],
    'Numeric2':[2, 5, 7]
}

In [3]:
df = pd.DataFrame(data=data)
df

,Click,Advertiser,Publisher,Numeric,Numeric2
0,0,Nike,CNN,10,2
1,1,ESPN,BBC,20,5
2,0,Nike,BBC,80,7


In [4]:
target='Click'
numerics=['Numeric', 'Numeric2']
categories=['Advertiser', 'Publisher']

In [5]:
# Initialise fields and variables encoder
encoder = {"currentcode": 0,  # Unique index for each numerical field or categorical variables
           "catdict": {},  # Dictionary that stores numerical and categorical variables
           "catcodes": {}}  # Dictionary that stores index for each categorical variables per categorical field

In [6]:
encoded =_convert_to_ffm(
    path='',
    df=df,
    type='sample',
    target=target,
    numerics=numerics,
    categories=categories,
    encoder=encoder
)

convert_to_ffm - START
UPDATING CATDICT: numeric field - Numeric
UPDATING CATDICT: numeric field - Numeric2
UPDATING CATDICT: categorical field - Advertiser
UPDATING CATDICT: categorical field - Publisher
UPDATING CATCODES: categorical field - Advertiser
UPDATING CATCODES: categorical value for field Advertiser - Nike
UPDATING CATCODES: categorical field - Publisher
UPDATING CATCODES: categorical value for field Publisher - CNN
UPDATING CATCODES: categorical value for field Advertiser - ESPN
UPDATING CATCODES: categorical value for field Publisher - BBC


In [7]:
encoded

{'currentcode': 4,
 'catdict': {'Numeric': 0, 'Numeric2': 0, 'Advertiser': 1, 'Publisher': 1},
 'catcodes': {'Advertiser': {'Nike': 1, 'ESPN': 3},
  'Publisher': {'CNN': 2, 'BBC': 4}}}

## Understanding `fields` , `features` and `value`

In [8]:
df

,Click,Advertiser,Publisher,Numeric,Numeric2
0,0,Nike,CNN,10,2
1,1,ESPN,BBC,20,5
2,0,Nike,BBC,80,7


In [9]:
data = {
    'Click' : [0, 1, 0],
    '0': [10, 20, 80],
    '1':[2, 5, 7],
    '2': ['Nike', 'ESPN', 'Nike'],
    '3': ['CNN', 'BBC', 'BBC'],
}

# fields are represented by Adertiser--0, Publisher--1, Numeric--2, Numeric2-3

In [10]:
df = pd.DataFrame(data)
df

,Click,0,1,2,3
0,0,10,2,Nike,CNN
1,1,20,5,ESPN,BBC
2,0,80,7,Nike,BBC


In [11]:
data = {
    'Click' : [0, 1, 0],
    'Numeric': [10, 20, 80],
    'Numeric2':[2, 5, 7],
    'Advertiser-Nike': [1, 0, 1],
    'Publisher-CNN': [1, 0, 0],
    'Advertiser-ESPN': [0, 1, 0],
    'Publisher-BBC': [0, 1, 1]
}

In [12]:
df = pd.DataFrame(data)
df

,Click,Numeric,Numeric2,Advertiser-Nike,Publisher-CNN,Advertiser-ESPN,Publisher-BBC
0,0,10,2,1,1,0,0
1,1,20,5,0,0,1,1
2,0,80,7,1,0,0,1


## features are represented by Numeric--0, Numeric--1, Advertiser-Nike-->2, Publisher-CNN-->3, Advertiser-ESPN-->4, Publisher-BBC-->5

In [13]:
df.columns = ['target', 0, 1, 2, 3, 4, 5]
df

,target,0,1,2,3,4,5
0,0,10,2,1,1,0,0
1,1,20,5,0,0,1,1
2,0,80,7,1,0,0,1


## LIBFFM format 
 `<label> <field>:<feature>:<value> <field>:<feature>:<value> <field>:<feature>:<value> ...`

### `0 0:0:10 1:1:2 2:2:1 3:3:1 `
### `1 0:0:20 1:1:5 2:4:1 3:5:1 `
### `0 0:0:80 1:1:7 2:2:1 3:5:1 `